In [1]:
#Slef-made Function
from Dataload import dataload
#from module import transformer,video_model

from module import video_model_attention
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
from train import methods


#Torch Library

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

#Sub tools
import pandas as pd

import os
import matplotlib.pylab as plt

#Util Library
import time
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid  

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video_pt=os.path.join(EMOTIPATH,"pt","Train")
Train_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Train")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"Train_audio.csv")

Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video_pt=os.path.join(EMOTIPATH,"pt","Valid")
Val_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Valid")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"Val_audio.csv")


train_table = pd.read_csv(Train_label,delimiter=' ')
val_table = pd.read_csv(Val_labels,delimiter=' ')
print(train_table['Label'].value_counts())
print(val_table['Label'].value_counts())



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)



frame_num=10

train_data_pt=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video_pt,face_path=Train_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Train)
valid_data_pt=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video_pt,face_path=Val_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Valid)





def load_pretrained_model(pre_train):
    face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_face.h")
    
   # face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_facev2.h")
    audio_model=video_model_attention.AudioRecognition(softmax=pre_train)
    audio_model.load_state_dict(torch.load("pre_trained_model/pre_embedded_audio.pth"),strict=False)
    image_model=video_model_attention.Video_modeller(frame_num,face_model=face_model,pre_train=pre_train)
    image_model.load_state_dict(torch.load("image_train_v2/image_train_15.pth"),strict=False)
    return(image_model,audio_model)



def show_img(dataset,index,frame):
    x=dataset[index][0][frame].cpu().numpy()
    plt.figure(1,(15,15))
    plt.axis('off')
    image = (x*0.5+0.5).transpose((1, 2, 0)).squeeze()
    plt.imshow(image)
    
    fig = plt.figure(2,(15,15))
   # plt.axis('off')
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(1,5),
                     axes_pad=0.1,
                     )
    for i in range(5):
        face=dataset[index][1][frame].cpu().numpy()
        image = (face[i,:]*0.5+0.5).transpose((1, 2, 0)).squeeze()
        grid[i].imshow(image,cmap='gray',interpolation='none')
        
img_model,audio_model=load_pretrained_model(False)
model=video_model_attention.video_transformer(img_model,audio_model)


3    932
2    923
1    806
Name: Label, dtype: int64
1    299
2    281
3    186
Name: Label, dtype: int64


In [2]:


weights=[932/932,932/923,932/806]
class_weights = torch.FloatTensor(weights).to(device)

train_dataloader = DataLoader(train_data_pt, batch_size=32
                       , num_workers=0,shuffle=True)
valid_dataloader = DataLoader(valid_data_pt, batch_size=32
                   , num_workers=0)
#model=video_model.video_transformer(img_model,audio_model)
model=model.to(device)

num_epochs=50
criterion = nn.CrossEntropyLoss(weight=class_weights)  

for name, child in model.named_children():
    if not name in ['face_model','frame_model' ]:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
               param.requires_grad = False

video_model is unfrozen
audio_model is unfrozen
fc1 is unfrozen
fc2 is unfrozen
relu is unfrozen


In [ ]:

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))


model_name="v4_attention"

print(model_name)

methods.full_train("v4_model",num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion,device)


v4_attention
Epoch:  1 

Validation 

Validation Loss:  1.0965209652980168
Validation Accuracy:  0.381201044386423
Confusion Matrix: 
 [[145 120  34]
 [140 117  24]
 [124  32  30]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.9153034090995789
Batch:  80 / 84
Batch Recognition loss:  1.0066038370132446

Average_Loss:  0.9368950447865895
Average_Accuracy:  0.6279594137542277
Confusion Matrix: 
 [[394 205 207]
 [128 645 150]
 [133 167 632]]
Epoch:  2 

Validation 

Validation Loss:  1.0538938790559769
Validation Accuracy:  0.46866840731070497
Confusion Matrix: 
 [[ 98 114  87]
 [ 72 160  49]
 [ 31  54 101]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8597193360328674
Batch:  80 / 84
Batch Recognition loss:  0.9309583902359009

Average_Loss:  0.89076138819967
Average_Accuracy:  0.6565201052236002
Confusion Matrix: 
 [[438 180 188]
 [135 644 144]
 [130 137 665]]
Epoch:  3 

Validation 

Validation Loss:  1.0587230895956357
Validation Accuracy:  0.46736292428198434
Confusion Matrix: 
 [[102 104  93]
 [ 72 152  57]
 [ 31  51 104]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8649159669876099
Batch:  80 / 84
Batch Recognition loss:  1.0439327955245972

Average_Loss:  0.8753661335933776
Average_Accuracy:  0.6621570838030816
Confusion Matrix: 
 [[457 182 167]
 [142 650 131]
 [132 145 655]]
Epoch:  4 

Validation 

Validation Loss:  1.0528745080033939
Validation Accuracy:  0.47389033942558745
Confusion Matrix: 
 [[105 114  80]
 [ 69 158  54]
 [ 32  54 100]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8312543034553528
Batch:  80 / 84
Batch Recognition loss:  1.0222537517547607

Average_Loss:  0.8730565522398267
Average_Accuracy:  0.6700488538143555
Confusion Matrix: 
 [[469 169 168]
 [135 640 148]
 [111 147 674]]
Epoch:  5 

Validation 

Validation Loss:  1.0494604880611103
Validation Accuracy:  0.4804177545691906
Confusion Matrix: 
 [[102 115  82]
 [ 66 161  54]
 [ 29  52 105]]
Train 



...

Batch:  40 / 84
Batch Recognition loss:  0.9167807698249817
Batch:  80 / 84
Batch Recognition loss:  0.7913594841957092

Average_Loss:  0.8480708279779979
Average_Accuracy:  0.6899661781285231
Confusion Matrix: 
 [[497 157 152]
 [137 650 136]
 [119 124 689]]
Epoch:  30 

Validation 

Validation Loss:  1.0411730607350667
Validation Accuracy:  0.489556135770235
Confusion Matrix: 
 [[107 103  89]
 [ 66 157  58]
 [ 26  49 111]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8838581442832947
Batch:  80 / 84
Batch Recognition loss:  1.0248360633850098

Average_Loss:  0.8501716412249065
Average_Accuracy:  0.6903419767004886
Confusion Matrix: 
 [[501 157 148]
 [139 654 130]
 [125 125 682]]
Epoch:  31 

Validation 

Validation Loss:  1.042866160472234
Validation Accuracy:  0.489556135770235
Confusion Matrix: 
 [[111 103  85]
 [ 74 152  55]
 [ 28  46 112]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.8067657947540283
Batch:  80 / 84
Batch Recognition loss:  0.9426125884056091

Average_Loss:  0.8545809068850109
Average_Accuracy:  0.685456595264938
Confusion Matrix: 
 [[495 153 158]
 [149 639 135]
 [114 128 690]]
Epoch:  32 

Validation 

Validation Loss:  1.0421347866455715
Validation Accuracy:  0.49216710182767626
Confusion Matrix: 
 [[ 94 116  89]
 [ 57 168  56]
 [ 26  45 115]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.7909013628959656
Batch:  80 / 84
Batch Recognition loss:  0.7839218378067017

Average_Loss:  0.853480359628087
Average_Accuracy:  0.6922209695603156
Confusion Matrix: 
 [[511 148 147]
 [131 644 148]
 [120 125 687]]
Epoch:  33 

Validation 

Validation Loss:  1.0381359482804935
Validation Accuracy:  0.4908616187989556
Confusion Matrix: 
 [[114 100  85]
 [ 76 152  53]
 [ 32  44 110]]
Train 



Batch:  40 / 84
Batch Recognition loss:  0.9427602291107178
